## DEVOUR Virtual Data Structure
Dynamic: The structure of the output DEVOUR graph is dynamically generated from raw input data

Explicit: Every node in a DEVOUR graph is explicitly typed and namespaced for reliability and interoperability between programs

Virtual: DEVOUR graphs contain their own serialized virtual memory space

Ordered: A DEVOUR graph's virtual memory is structured as an ordered list for performance

Unique: A DEVOUR graph contains only one instance of a given "real" value, mapped into its virtual memory

Relational: A DEVOUR graph's virtual memory is referenced via indexed pointers within its data structure

In [98]:
class Graph:
    def __init__(self):
        self.graph = []
        self.cache = {
            "shape": [],
            "memory": []
        }

    # adds a new object to the graph
    def add(self, node):
        # add to graph
        self.graph.append(node)

    def redirect_node(self, node):
        shape = self.cache["shape"]
        memory = self.cache["memory"]

        # for dictionaries, add each member node
        if type(node) == dict:
            # cache keys and values
            new_dict = {}
            for key, value in node.items():
                if type(value) == dict:
                    value = self.redirect_node(value)

                # add dict key to memory
                if key not in memory:
                    memory.append(key)
                    key_index = "#" + hex(len(memory) - 1)
                else:
                    key_index = "#" + hex(memory.index(key))

                # add dict value to memory
                if value not in memory:
                    memory.append(value)
                    value_index = "#" + hex(len(memory) - 1)
                else:
                    value_index = "#" + hex(memory.index(value))

                new_dict[key_index] = value_index

            # check if the dict is in memory already
            if new_dict in memory:
                node = "#" + hex(memory.index(new_dict))
            else:
                node = new_dict

        # redirect object to virtual memory
        if type(node) == str:
            is_pointer = node[0] == "#"
        else:
            is_pointer = False
        
        if not is_pointer:
            if node not in memory:
                memory.append(node)
                value_pointer = "#" + hex(len(memory) - 1)
            else:
                value_pointer = "#" + hex(memory.index(node))

        # add a redirector to shape
        shape.append(value_pointer)

        # return a pointer to the node
        return "#" + hex(len(memory) - 1)

    # exports the graph as a single dictionary object
    def serialize(self):
        # serialize DevourObject instances
        for node in self.graph:
            self.redirect_node(node)

        return self.cache

In [99]:
# test
graph = Graph()
graph.add(1.0)
graph.add(1.0)
graph.add(3.0)
graph.add(3.0)
graph.add(1.0)
graph.add({
        "name": "TestObject",
        "visible": True
    })
graph.add({
        "name": "TestObject",
        "visible": True
    })
graph.add({
        "name": "TestObject",
        "visible": True,
        "transform": {
            "position": [0.0, 0.0, 0.0],
            "rotation": [0.0, 0.0, 0.0],
            "scale": [1.0, 1.0, 1.0]
        }
    })
graph.add({
        "name": "TestObject",
        "visible": True,
        "transform": {
            "position": [0.0, 0.0, 0.0],
            "rotation": [0.0, 0.0, 0.0],
            "scale": [1.0, 1.0, 1.0]
        }
    })

# write to disk
import json
with open("DEVOURTest.json", "w") as outfile:
    json.dump(graph.serialize(), outfile, indent = 4)